<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Refactoring/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Załadowanie repozytorium

##Załadowanie repozytorium

In [ ]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

In [ ]:
!git checkout Refactoring

#Setup sparka

# Załadowanie nowych baz

##Setup sparka

In [ ]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

In [ ]:
import pandas as pd
from pyspark.sql.functions import split, explode, collect_list, first, array_distinct, min, max
from google.colab import drive
from lib.pyspark_init import create_spark_context, load_data

In [ ]:
spark = create_spark_context()
spark

#Pobranie danych

##Import danych

In [ ]:
!chmod +x download_data.sh
!./download_data.sh

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

## Wybierzmy z tabel tylko istotne rekordy:
- df_title_basic: wybierzmy tylko rekordy z primaryProfession zawierającym "actor" lub "actress",
- df_title_principals: wybierzmy rekordy z category zawierającym "actor" lub "actress".

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - startYear (data wypuszczenia produkcji - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - endYear (data zakończenia serii, w innych przypadkach \N - można wykorzystać do porównania epok filmów, w których grali aktorzy)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - knownForTitles (tytuły produkcji, z których znany jest aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [ ]:
%%time
data = load_data(spark)
data.show(3)

# Załadowanie nowych baz

   **W celu załadowania nowych danych należy:**


1. Pobrać token API:
  * Wejść w *zespół Aktorzy* na teams
   * Wybrać zakładkę *pliki*
    * Pobrać plik *kaggle.json* - API
2. Utworzyć plik na swoim dysku pod nazwą "Big Data"
3. Wrzucić pobrane API na swój dysk
4. Uruchomić kod :)




## Wczytajmy bazę oskarów: "Best Movies Watchlist"


In [ ]:
drive.mount('/content/gdrive')

#%cd /content/gdrive/MyDrive/Big Data/kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Big Data"

#from google.colab import drive
#drive.mount('/content/drive')
#random.seed(123456)
#np.set_printoptions(precision=6, suppress=True)

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle #Utwórz katalog o nazwie „.kaggle”
! cp kaggle.json ~/.kaggle/ # Skopiuj plik „kaggle.json” do tego nowego katalogu
! chmod 600 ~/.kaggle/kaggle.json #Przydziel wymagane uprawnienia do tego pliku.

In [ ]:
#import pandas as pd

! kaggle datasets download unanimad/the-oscar-award #załadujmy dane
! unzip the-oscar-award.zip #rozpakujmy z zip'a
oscars=spark.read.option("header","true").csv('the_oscar_award.csv')
#oscars=pd.read_csv('the_oscar_award.csv')
oscars.show(5)


Main Data:
* primaryName	- aktor
* originalTitle - nazwa filmu

W nowej bazie oskarów:
* name - aktor
* film - film

## Wczytajmy bazę złotych globów: "Golden Globe Awards, 1944 - 2020"

In [ ]:
! kaggle datasets download unanimad/golden-globe-awards #załadujmy dane
! unzip golden-globe-awards.zip #rozpakujmy z zip'a
globe=spark.read.option("header","true").csv('golden_globe_awards.csv')
globe.show(5)

## Wczytajmy bazę złotych globów: "Primetime Emmy Awards, 1949-2017"

In [ ]:
! kaggle datasets download unanimad/emmy-awards #załadujmy dane
! unzip emmy-awards.zip #rozpakujmy z zip'a
emmy_awards_category=spark.read.option("header","true").csv('emmy_awards_categories.csv')
emmy_awards_category.show(5)


In [ ]:
emmy_awards=spark.read.option("header","true").csv('the_emmy_awards.csv')
emmy_awards.show(5)

## Zaczytanie danych TMDB 

   informacje o aktorach i osobach pracujących przy produkcji filmów oraz budżetach, gatunkach, popularności, wytwórniach filmowych, oryginalnyh językach i tytułach


In [ ]:
#!kaggle kernels init -p tmdb/tmdb-movie-metadata #wygenerować metadane
! kaggle datasets download tmdb/tmdb-movie-metadata #załadujmy dane
! unzip tmdb-movie-metadata.zip
tmdb_credits=spark.read.option("header","true").csv('tmdb_5000_credits.csv')
tmdb_movies=spark.read.option("header","true").csv('tmdb_5000_movies.csv')

In [ ]:
tmdb_credits.show(5)

In [ ]:
tmdb_movies.show(5)

# Połączmy dane na temat nagród z głównymi danymi

Wybierzmy tylko rekordy z interesujących nas kategorii

In [ ]:
oscars_selected = oscars.filter((oscars.category.like('%ACTOR%')) | (oscars.category.like('%ACTRESS%')))
globe_selected = globe.filter((globe.category.like('%Actor%')) | (globe.category.like('%Actress%')))

In [ ]:
actor_categories = ['Outstanding Lead Actor in a Comedy Series', 'Outstanding Lead Actor in a Drama Series', 'Outstanding Lead Actor in a Limited or Anthology Series or Movie', 'Outstanding Lead Actress in a Comedy Series', 'Outstanding Lead Actress in a Drama Series', 'Outstanding Lead Actress in a Limited or Anthology Series or Movie', 'Outstanding Supporting Actor in a Comedy Series', 'Outstanding Supporting Actor in a Drama Series', 'Outstanding Supporting Actor in a Limited or Anthology Series or Movie', 'Outstanding Supporting Actress in a Comedy Series', 'Outstanding Supporting Actress in a Drama Series', 'Outstanding Supporting Actress in a Limited or Anthology Series or Movie', 'Outstanding Character Voice-Over Performance', 'Outstanding Guest Actor in a Drama Series', 'Outstanding Guest Actor in a Comedy Series', 'Outstanding Guest Actress in a Drama Series', 'Outstanding Guest Actress in a Comedy Series', 'Outstanding Narrator', 'Outstanding Actor in a Short Form Comedy or Drama Series', 'Outstanding Actress in a Short Form Comedy or Drama Series', 'Best Specialty Act – Single or Group', 'Outstanding Voice-Over Performance', 'Outstanding Sports Personality', 'Most Outstanding Live Personality', 'Most Outstanding Kinescoped Personality']
emmy_awards_selected = emmy_awards.filter(emmy_awards.category.isin(actor_categories))

Przygotujmy dane z tabeli emmy_awards_selected do połączenia (wybieramy tylko imię i nazwisko aktora z kolumny staff)

In [ ]:
emmy_awards_selected = emmy_awards_selected.withColumn('staff',explode(split('staff',', ')))
emmy_awards_selected = emmy_awards_selected.groupby('nominee', 'staff').agg(first('id').alias('id'), first('year').alias('year'), first('category').alias('category'), first('nominee').alias('nominee'), first('staff').alias('staff'), first('company').alias('company'), first('producer').alias('producer'), first('win').alias('win'))

Wybierzmy tylko kolumny, które nas interesują z danych na temat nagród

In [ ]:
oscars_selected = oscars.select(["year_ceremony", "category", "name", "film", "winner"])
globe_selected = globe.select(["year_award", "category", "nominee", "film", "win"])
emmy_awards_selected = emmy_awards.select(["year", "category", "nominee", "staff", "company", "producer", "win"])

Zmieńmy nazwy kolumn, aby wiadomo było po połączeniu, któe dane odnoszą się do których nagród

In [ ]:
oscars_selected = oscars_selected.withColumnRenamed("year_ceremony","year_oscars").withColumnRenamed("category","category_oscars").withColumnRenamed("film","film_oscars").withColumnRenamed("winner","winner_oscars")

globe_selected = globe_selected.withColumnRenamed("year_award","year_globes").withColumnRenamed("category","category_globes").withColumnRenamed("film","film_globes").withColumnRenamed("win","win_globes")

emmy_awards_selected = emmy_awards_selected.withColumnRenamed("year","year_emmy").withColumnRenamed("category","category_emmy").withColumnRenamed("nominee","nominee_emmy").withColumnRenamed("company","company_emmy").withColumnRenamed("producer","producer_emmy").withColumnRenamed("win","win_emmy")

Złączmy dane dotyczące nagród z głównymi danymi

In [ ]:
print("data dataframe size: ", (data.count(), len(data.columns)))
print("oscars_selected dataframe size: ", (oscars_selected.count(), len(oscars_selected.columns)))
print("globe_selected dataframe size: ", (globe_selected.count(), len(globe_selected.columns)))
print("emmy_awards_selected dataframe size: ", (emmy_awards_selected.count(), len(emmy_awards_selected.columns)))
data = data.join(oscars_selected, data.primaryName == oscars_selected.name, "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(globe_selected, data.primaryName == globe_selected.nominee, "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(emmy_awards_selected, data.primaryName == emmy_awards_selected.staff, "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

In [ ]:
data = data.groupby('nconst').agg(first('tconst').alias("tconst"), first('titleType').alias("titleType"), first('originalTitle').alias("originalTitle"), first('isAdult').alias("isAdult"), first('startYear').alias("startYear"), first('endYear').alias("endYear"), first('genres').alias("genres"), first('category').alias("category"), first('characters').alias("characters"), first('primaryName').alias("primaryName"), first('knownForTitles').alias("knownForTitles"), collect_list('year_oscars').alias("year_oscars"), collect_list('category_oscars').alias("category_oscars"), collect_list('film_oscars').alias("film_oscars"), collect_list('winner_oscars').alias("winner_oscars"), collect_list('year_globes').alias("year_globes"), collect_list('category_globes').alias("category_globes"), collect_list('film_globes').alias("film_globes"), collect_list('win_globes').alias("win_globes"), collect_list('year_emmy').alias("year_emmy"), collect_list('category_emmy').alias("category_emmy"), collect_list('nominee_emmy').alias("nominee_emmy"), collect_list('company_emmy').alias("company_emmy"), collect_list('producer_emmy').alias("producer_emmy"), collect_list('win_emmy').alias("win_emmy"))

Pokażmy kilka przykłądowych wierszy ostatecznej tabeli

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
sample = pd.DataFrame(sample, columns = data.columns)
sample